# What's this??
Seleniumを使ってATBBからデータを引っ張ってくる

http://atbb.athome.jp/

In [1]:
# import libraries
import time
import math
import pandas as pd
import numpy as np
import os
import urllib.request

In [2]:
# import selenium libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# import original libraries
from utils.python import atbb

In [4]:
# variables
path = '/Users/ryusukekimura/Google ドライブ/github/demonstration/rpa/chrome-driver-79.0.3945.36/chromedriver'
url = 'http://atbb.athome.jp/'
timeout = 30

In [5]:
# create output directory
dir_data = './data/atbb/'
if not os.path.isdir(dir_data):
    os.makedirs(dir_data)
dir_img = './img/atbb/'
if not os.path.isdir(dir_img):
    os.makedirs(dir_img)

In [6]:
# define output columns
cols =  ['fetch_datetime', 'todofuken', 'shikuchoson', 'bld_name', 'room_name', 'room_madori', 'bld_address', 'access', 'num_rooms',
              'chinryo', 'kanrihi', 'sikikin', 'reikin', 'kibo_kozo', 'menseki_tsubo', 'syunko_nengappi', 'keiyaku_kikan', 'kousinryo',
               'torihiki_taiyo', 'parking', 'hoken', 'kanri_gaisya', 'kagi_koukan', 'kouza_furikae', 'hosyo_gaisya', 'setsubi', 'bikou']

In [23]:
# open login page
driver = webdriver.Chrome(executable_path=path)
driver.get(url)

In [12]:
# login
## read id and password
id, pw = atbb.read_id_pass()

## input id and password
driver.find_element_by_id('loginFormText').send_keys(id)
driver.find_element_by_id('passFormText').send_keys(pw)

## click login button
driver.find_element_by_xpath('//*[@id="loginSubmit"]').click()

In [13]:
# click 物件検索
atbb.wait_driver(driver, timeout, 'header')
driver.find_element_by_xpath('//*[@id="header"]/header-menu/div/nav/ul/li[1]/a').click()

# click 成約事例検索
atbb.wait_driver(driver, timeout, 'header')
driver.find_element_by_xpath('//*[@id="header"]/header-menu/div/nav/ul/li[1]/div/form/div/div[3]/div').click()

In [14]:
# switch window
driver.switch_to.window(driver.window_handles[-1])

In [15]:
# 所在地検索
## click radio button 賃貸居住用
xpath = '/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form/table[2]/tbody/tr[2]/td/div[1]/label/input'
driver.find_element_by_xpath(xpath).click()

## click checkbox 東京
xpath = '//*[@id="checkZenTodofuken"]/table/tbody/tr[3]/td[2]/table/tbody/tr[1]/td[1]/label/input'
driver.find_element_by_xpath(xpath).click()

## 検索
xpath = '//*[@id="searcharea"]/div[1]/table[2]/tbody/tr[2]/td/div/table/tbody/tr/td[1]/input'
driver.find_element_by_xpath(xpath).click()

In [16]:
# 市区町村の選択
xpath = '//*[@id="sentaku3ZenShikugun_13"]/option[2]'
button = driver.find_element_by_xpath(xpath)
driver.execute_script('arguments[0].click();', button)

# 検索条件指定画面へ
xpath = '/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form/div[4]/input[1]'
driver.find_element_by_xpath(xpath).click()

In [17]:
driver.find_element_by_xpath('/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form/table/tbody/tr[2]/td[2]/text()[3]').click()

InvalidSelectorException: Message: invalid selector: The result of the xpath expression "/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form/table/tbody/tr[2]/td[2]/text()[3]" is: [object Text]. It should be an element.
  (Session info: chrome=80.0.3987.116)


In [19]:
# 検索結果をガーッと取ってくる
## 賃貸マンションを選択
driver.find_element_by_xpath('/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form/table/tbody/tr[1]/td[2]/input[1]').click()

## 駅徒歩10分以内を選択
driver.find_element_by_xpath('/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form/table/tbody/tr[2]/td[2]/input[5]').click()

## 2DK/2LDK
driver.find_element_by_xpath('/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form/table/tbody/tr[4]/td[2]/table/tbody/tr[2]/td[3]/input').click()
driver.find_element_by_xpath('/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form/table/tbody/tr[4]/td[2]/table/tbody/tr[2]/td[2]/input').click()

## 築年数10年以内
driver.find_element_by_xpath('/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form/table/tbody/tr[7]/td[2]/input[8]').click()

# 検索
driver.find_element_by_xpath('/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form/div[1]/div/input[1]').click()

In [20]:
# 物件詳細へ
driver.find_element_by_xpath('//*[@id="shosai_0"]').click()

In [21]:
# extract information
xpath = '/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form[2]/table[4]/tbody/tr[1]/td[2]/span'
chinryo_touroku = driver.find_element_by_xpath(xpath).text

xpath = '/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form[2]/table[4]/tbody/tr[1]/td[4]'
heibei_tanka = driver.find_element_by_xpath(xpath).text

xpath = '/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form[2]/table[4]/tbody/tr[1]/td[6]'
koukaibi = driver.find_element_by_xpath(xpath).text

xpath = '/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form[2]/table[4]/tbody/tr[2]/td[2]/span'
chinryo_seiyaku = driver.find_element_by_xpath(xpath).text

xpath = '/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form[2]/table[4]/tbody/tr[2]/td[4]'
heibei_tanka_seiyaku = driver.find_element_by_xpath(xpath).text

xpath = '/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form[2]/table[4]/tbody/tr[2]/td[6]'
seiyakubi = driver.find_element_by_xpath(xpath).text

xpath = '/html/body/table/tbody/tr[3]/td/table/tbody/tr[1]/td[3]/form[2]/table[5]/tbody/tr[1]/td[2]'
bukken_syumoku = driver.find_element_by_xpath(xpath).text

In [22]:
array = [chinryo_touroku, heibei_tanka, koukaibi, chinryo_seiyaku, heibei_tanka_seiyaku, seiyakubi, bukken_syumoku]
print(array)

['25.80万円', '0.46万円/㎡ ', '2019/06/11 ', '25.80万円', '0.46万円/㎡ ', '2019/06/29 ', '貸マンション ']
